In [2]:
import pandas as pd

In [7]:
from os import error
df = pd.read_csv('kaggle_fake_train.csv', engine = 'python', on_bad_lines='skip')

In [8]:
df.head()
#1, Fakenews
#0, Not a Fakenews

#take Title and find fakenews or not creating LSTM model

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
#Drop nan Values , Drop nan values when we find it.
df = df.dropna()


In [10]:
## Get independent features

X = df.drop('label', axis = 1)

In [11]:
## Get dependent features
y = df['label']

In [12]:
X.shape

(9615, 4)

In [13]:
y.shape

(9615,)

In [14]:
import tensorflow as tf

In [15]:
#Emb means word2vec
#pre padding and post padding >>  add 0
# Sequntial
# OHE >> on which index word is in


from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
## Vocabulary Size >> Get how many unique words in Corpus
Voc_Size=5000


In [17]:
# One hot Rep

messages=X.copy()

In [18]:
#use title only for this FAKE NEWS
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages.reset_index(inplace=True)

In [20]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
9610,10948,10948,"U.S., in Reversal, Issues Permit for Keystone ...",Clifford Krauss,"HOUSTON — During his presidential campaign,..."
9611,10949,10949,University of Chicago: Why Milo Scares Student...,Tom Ciccotta,An article published by the University of Chic...
9612,10950,10950,Migrants FLOOD Into U.S. From Mexico Right Bef...,Robert Gehl,"With 12 days to go until the election, all eye..."
9613,10951,10951,Twitter Temporarily Censored Drudge Report Twe...,Allum Bokhari,"Earlier today, users on Twitter reported that ..."


In [21]:


# 1st step >> Stop words

import nltk
import re
from nltk.corpus import stopwords


In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
# 2nd DATASET Preprocessing
# Stemming
# And then Stopwords

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
# Give One hot representation

onehot_rep = [one_hot(words, Voc_Size) for words in corpus]
onehot_rep

[[1654, 4318, 1949, 2519, 75, 777, 802, 1788, 3183, 4394],
 [195, 1426, 3043, 4640, 1117, 3124, 1791],
 [3187, 3826, 1959, 3272],
 [3501, 2634, 583, 2796, 94, 4779],
 [4147, 1117, 181, 2968, 875, 3129, 1117, 513, 3087, 375],
 [666,
  3701,
  2398,
  1778,
  608,
  774,
  4791,
  137,
  2039,
  2499,
  1261,
  1437,
  3222,
  377,
  1791],
 [3513, 57, 1306, 3636, 1077, 1846, 4949, 2965, 2850, 3011, 581],
 [4240, 2428, 1376, 4281, 2761, 4026, 774, 1588, 2850, 3011, 581],
 [3270, 2397, 3977, 72, 2515, 4819, 2043, 222, 774, 431],
 [2317, 4990, 371, 2600, 3655, 2132, 3893, 4033],
 [3320, 147, 3061, 987, 3498, 3049, 1215, 3486, 2571, 2769, 4365],
 [2796, 3103, 75, 4819, 774, 2761],
 [2477, 1377, 3668, 630, 3216, 2118, 394, 4896, 4126],
 [4405, 693, 3409, 1685, 2536, 4496, 3060, 2850, 3011, 581],
 [186, 1553, 3630, 2105, 3325, 2850, 3011, 581],
 [3586, 1110, 1277, 2207, 1436, 1682, 3857, 1073, 24, 3022],
 [43, 2002, 1426],
 [336, 3359, 3569, 621, 774, 4859, 678, 1791],
 [1695, 578, 3043, 1630

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [27]:
onehot_rep[1]

[195, 1426, 3043, 4640, 1117, 3124, 1791]

In [28]:
# Take Onehot rep and apply to Embbdeing layer (Word2Vec) convert to vectors
# Same length so using padding on pre. sentence length and padding 0 before it as pre.

sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1788 3183 4394]
 [   0    0    0 ... 1117 3124 1791]
 [   0    0    0 ... 3826 1959 3272]
 ...
 [   0    0    0 ... 3399 4553 4836]
 [   0    0    0 ... 4654 1176 1791]
 [   0    0    0 ... 2850 3011  581]]


In [30]:
embedded_docs[0]
# len 20 and adding pre padding , Post means end it will 0, make sure all lenth ging to be 20.

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1654,
       4318, 1949, 2519,   75,  777,  802, 1788, 3183, 4394], dtype=int32)

In [31]:
#Creating a model, Apply Embedding layer , and Apply LSTM layer.

embedding_vector_features = 40 # every index will represent by 40 features or values.
model = Sequential()
#add embedding layer.
model.add(Embedding(Voc_Size, embedding_vector_features, input_length=sent_length))
#Apply LSTM with 100 neurons
model.add(LSTM(100))
#dense as 1 means out will binary , So we use sigmoid act, binary crossentropy and adam optimizers.
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [32]:
len(embedded_docs), y.shape

(9615, (9615,))

In [33]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [35]:
## Model Training Final

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.7649 - loss: 0.4986 - val_accuracy: 0.9014 - val_loss: 0.2194
Epoch 2/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.9271 - loss: 0.1695 - val_accuracy: 0.9162 - val_loss: 0.2032
Epoch 3/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.9678 - loss: 0.0906 - val_accuracy: 0.9114 - val_loss: 0.2145
Epoch 4/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9768 - loss: 0.0678 - val_accuracy: 0.9042 - val_loss: 0.3160
Epoch 5/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.9923 - loss: 0.0290 - val_accuracy: 0.9055 - val_loss: 0.3540
Epoch 6/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.9945 - loss: 0.0183 - val_accuracy: 0.8928 - val_loss: 0.3504
Epoch 7/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.9971 - loss: 0.0154 - val_accuracy: 0.9042 - val_loss: 0.4789
Epoch 8/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9987 - loss: 0.0067 - val_accu

In [36]:
#performance metrics and accuracy

y_pred = model.predict(X_test)

100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [40]:
y_pred = np.where(y_pred>0.6,1,0)

In [41]:
from sklearn.metrics import confusion_matrix


In [42]:
confusion_matrix(y_test, y_pred)

array([[1642,  158],
       [ 194, 1179]])

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8890639773085408

In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1800
           1       0.88      0.86      0.87      1373

    accuracy                           0.89      3173
   macro avg       0.89      0.89      0.89      3173
weighted avg       0.89      0.89      0.89      3173

